In [25]:
%pip install selenium


Note: you may need to restart the kernel to use updated packages.


In [257]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [343]:
def click_show_more(driver):
    try:
        show_more_button = driver.find_element(By.XPATH, '//button[@class="button_Button__meEg5 button-base_Button__9SPjH"]')
        show_more_button.click()
        time.sleep(1)  # Adjust as needed
        print('Show More button clicked')
    except NoSuchElementException:
        print("Show More button not found or clicked")
    except ElementClickInterceptedException:
        print("Element Click Intercepted while clicking Show More button")

In [348]:
def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    
    '''Gathers jobs as a dataframe, scrape from Glassdoor'''
    
    #Initializing the webdriver
    
    service = Service(executable_path= path)
    options = webdriver.ChromeOptions()
  
    driver = webdriver.Chrome(service=service, options=options)
    #driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    

    url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    
    driver.get(url)
    jobs = []
    job_count=0
    last_processed_index=-2
    df = pd.DataFrame()

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.
        time.sleep(slp_time)
        try:
            sign_up_popup = driver.find_element(By.XPATH, '//div[@class="Modal"]')
            if sign_up_popup.is_displayed():
                close_button = sign_up_popup.find_element(By.XPATH, '//button[@class="CloseButton"]')
                close_button.click()
        except NoSuchElementException:
            pass
        time.sleep(1)
        print('going through each job')
        #Going through each job in this page
        job_buttons = driver.find_elements(By.CLASS_NAME,"JobsList_jobListItem__JBBUV")
        print('current length of job buttons')
        print(len(job_buttons)) 
        

        for job_button in job_buttons[last_processed_index+2:-1]: 
            
            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break
             # Handle sign-up pop-up 
            try: 
                sign_up_popup = driver.find_element(By.XPATH, '//div[@class="Modal"]')
                if sign_up_popup.is_displayed():
                    close_button = sign_up_popup.find_element(By.XPATH, '//button[@class="CloseButton"]')
                    close_button.click()
            except NoSuchElementException:
                pass
            print('its closed')
            time.sleep(1)
            # Scroll the element into view
            #ActionChains(driver).move_to_element(job_button).perform()
            try:
                job_button.click() 
            except ElementClickInterceptedException:
                print("ElementClickInterceptedException occurred. Skipping this job.")
                continue

            #job_button.click() 
            collected_successfully = False
            attempts = 0 
            while not collected_successfully:
                print('entered not collecetd successfully')
                try:
 
                    company_name = driver.find_element(By.CSS_SELECTOR, '.JobDetails_jobDetailsHeader__qKuvs .EmployerProfile_employerName__Xemli').text
                    
                    #company_name = driver.find_element(By.XPATH, '//div[@class="EmployerProfile_employerInfo__GaPbq EmployerProfile_employerWithLogo__R_rOX"]//span[@class="EmployerProfile_employerName__Xemli"]').text
                    location = driver.find_element(By.XPATH,'.//div[@class="JobDetails_location__MbnUM"]').text
                   
                    job_title = driver.find_element(By.XPATH,'.//h1[contains(@class, "JobDetails_jobTitle__Rw_gn")]').text
                  
                    job_description = driver.find_element(By.XPATH,'.//div[@class="JobDetails_jobDescriptionWrapper__BTDTA JobDetails_jobDetailsSectionContainer__NMzYt"]').text
                    #job_description = driver.find_element(By.XPATH,'.//div[@class="JobDetails_jobDescription__6VeBn JobDetails_blurDescription__fRQYh"]').text
                    
                  
                    collected_successfully = True    
                
                except:
                        print('job not done, previous company')
                        time.sleep(5)  
                        #attempts+=1
            # if not collected_successfully:
            #     print('Job details could not be collected after 3 attempts. Skipping this job.')
            #     continue
               
            try:
                salary_estimate = driver.find_element(By.XPATH,'.//div[@class="SalaryEstimate_salaryEstimateNumber__7EPgD"]').text
                print(salary_estimate)
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating=driver.find_element(By.CSS_SELECTOR, '.JobDetails_jobDetailsHeader__qKuvs .EmployerProfile_ratingContainer__N4hxE').text
                #rating = driver.find_element(By.XPATH,'.//div[@class="EmployerProfile_ratingContainer__N4hxE"]').text
               
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."
                
            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            try:
                #size = driver.find_element(By.XPATH,'.//div[@class="JobDetails_overviewItemValue__5TqNi"]').text
                size=driver.find_element(By.CSS_SELECTOR, 'div.JobDetails_overviewItemValue__5TqNi').text
            except NoSuchElementException:
                size = -1

            try:
                founded = driver.find_element(By.XPATH,  "//div[@class='JobDetails_overviewItem__35s2T'][2]/div[@class='JobDetails_overviewItemValue__5TqNi']").text
                #//div[@class='grid-row'][2]/div[1]

            except NoSuchElementException:
                founded = -1

            try:
                type_of_ownership = driver.find_element(By.XPATH,  "//div[@class='JobDetails_overviewItem__35s2T'][3]/div[@class='JobDetails_overviewItemValue__5TqNi']").text
            except NoSuchElementException:
                type_of_ownership = -1

            try:
                industry = driver.find_element(By.XPATH,  "//div[@class='JobDetails_overviewItem__35s2T'][4]/div[@class='JobDetails_overviewItemValue__5TqNi']").text
            except NoSuchElementException:
                industry = -1

            try:
                sector = driver.find_element(By.XPATH,  "//div[@class='JobDetails_overviewItem__35s2T'][5]/div[@class='JobDetails_overviewItemValue__5TqNi']").text
            except NoSuchElementException:
                sector = -1

            try:
                    revenue = driver.find_element(By.XPATH,  "//div[@class='JobDetails_overviewItem__35s2T'][6]/div[@class='JobDetails_overviewItemValue__5TqNi']").text
            except NoSuchElementException:
                    revenue = -1

            if verbose:
                
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            })
            
            job_count += 1
            print("Total jobs collected so far:", job_count)
            last_processed_index+=1
            print("Total indexes collected so far:", last_processed_index)
            if job_count % 50 == 0:  # Save data after scraping every 50 jobs
                df = pd.DataFrame(jobs)
                df.to_csv('data_scraped.csv', index=False)

        
        print('exited the for loop')

        if job_count >= num_jobs:
           
            print("Reached the target number of jobs.")
            break
            
        else:
            print("No more jobs available in the current list")
             # Handle sign-up pop-up 
            try: 
                sign_up_popup = driver.find_element(By.XPATH, '//div[@class="Modal"]')
                if sign_up_popup.is_displayed():
                    close_button = sign_up_popup.find_element(By.XPATH, '//button[@class="CloseButton"]')
                    close_button.click()
            except NoSuchElementException:
                pass

            time.sleep(1)
            # Click the "Show More" button
            
            try:
                print('trying to click on show more button')

                show_more_button = driver.find_element(By.XPATH, '//button[@data-test="load-more"]')
                show_more_button.click()
                print("clicked on show more")
            except NoSuchElementException:
                print("No more jobs available.")
                break


    return pd.DataFrame(jobs)                         
#This line converts the dictionary object into a pandas DataFrame.

           
        

In [ ]:
path="C:/Users/dantu/Documents/salary_pred_project/chromedriver.exe"
df=get_jobs('data scientist', 1000, False, path, 15)